In [20]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# 01. Dataset loading

In [21]:
#Load the csv file
matches_df = pd.read_csv('matches_dataset_private.csv')

In [22]:
matches_df.head()

,match_number,division,date,match_time,match_pitch,local_team,visitor_team,local_team_player_01_jersey,local_team_player_01_front_row,local_team_player_01_captain,...,scoreboard_local_25_minute,scoreboard_local_26_type,scoreboard_local_26_player,scoreboard_local_26_minute,scoreboard_visitor_25_type,scoreboard_visitor_25_player,scoreboard_visitor_25_minute,scoreboard_visitor_26_type,scoreboard_visitor_26_player,scoreboard_visitor_26_minute
0,9,2A,11/01/2025,14:00,Parque Deportivo Puerta de Hierro,Aeronáuticos,BigMat Tabanera Lobos Segovia,1.0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,2A,10/11/2024,17:00,Parque Deportivo Puerta de Hierro,Aeronáuticos,Canal Red Vallecas Rugby Union,1.0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11,2A,26/01/2025,11:00,Parque Deportivo Puerta de Hierro,Aeronáuticos,Getafe Club de Rugby,1.0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7,2A,30/11/2024,13:00,Parque Deportivo Puerta de Hierro,Aeronáuticos,Industriales C SlashMobility Las Rozas,1.0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,2A,24/11/2024,12:00,Parque Deportivo Puerta de Hierro,Aeronáuticos,Rugby Unión Norte,1.0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
matches_df.loc[:, 'match_number':'visitor_team_player_23_initial_position'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Columns: 283 entries, match_number to visitor_team_player_23_initial_position
dtypes: float64(92), int64(139), object(52)
memory usage: 161.5+ KB


In [24]:
#Kepp license and jersey columns as floats since there is a combination of numeric values and non-number values (NaN)

jersey_columns = matches_df.filter(regex='player_\d{2}_jersey').columns
license_columns = matches_df.filter(regex='_license').columns
event_columns = matches_df.filter(regex='_\d{2}_player').columns #Scoreboards, substitutions and cards

#Example
print(matches_df[jersey_columns].dtypes)

local_team_player_01_jersey      float64
local_team_player_02_jersey      float64
local_team_player_03_jersey      float64
local_team_player_04_jersey      float64
local_team_player_05_jersey      float64
local_team_player_06_jersey      float64
local_team_player_07_jersey      float64
local_team_player_08_jersey      float64
local_team_player_09_jersey      float64
local_team_player_10_jersey      float64
local_team_player_11_jersey      float64
local_team_player_12_jersey      float64
local_team_player_13_jersey      float64
local_team_player_14_jersey      float64
local_team_player_15_jersey      float64
local_team_player_16_jersey      float64
local_team_player_17_jersey      float64
local_team_player_18_jersey      float64
local_team_player_19_jersey      float64
local_team_player_20_jersey      float64
local_team_player_21_jersey      float64
local_team_player_22_jersey      float64
local_team_player_23_jersey      float64
visitor_team_player_01_jersey    float64
visitor_team_pla

In [25]:
#Check for completely empty columns

df_columns = len(matches_df.columns)
df_columns_clean = len(matches_df.dropna(axis=1, how='all').columns)

print('Original number of columns: ', df_columns)
print ('Number of columns after cleaning: ', df_columns_clean)

print('Total number of columns to drop: ', df_columns - df_columns_clean)
print('Set of columns to be droped (player, type, minute: ', (df_columns - df_columns_clean)/3)

Original number of columns:  601
Number of columns after cleaning:  508
Total number of columns to drop:  93
Set of columns to be droped (player, type, minute:  31.0


In [26]:
matches_df.dropna(axis=1, how='all', inplace=True)

In [27]:
#Date parsing
#Parsing to date format the column 'date'

matches_df['date'] = pd.to_datetime(matches_df['date'], dayfirst=True)
matches_df['match_time'] = pd.to_datetime(matches_df['match_time'], format='%H:%M').dt.time

In [28]:
#Sort matches by date and hour
matches_df.sort_values(['date', 'match_time'], inplace=True)

#Reset index
matches_df.reset_index(drop=True, inplace=True)

In [29]:
matches_df.head()

,match_number,division,date,match_time,match_pitch,local_team,visitor_team,local_team_player_01_jersey,local_team_player_01_front_row,local_team_player_01_captain,...,scoreboard_local_23_minute,scoreboard_local_24_type,scoreboard_local_24_player,scoreboard_local_24_minute,scoreboard_local_25_type,scoreboard_local_25_player,scoreboard_local_25_minute,scoreboard_local_26_type,scoreboard_local_26_player,scoreboard_local_26_minute
0,1,2A,2024-10-05,13:00:00,Universidad Alfonso X el Sabio,C.R Majadahonda,Canal Red Vallecas Rugby Union,1.0,1,0,...,64.0,T,10.0,64.0,E,10.0,68.0,E,9.0,70.0
1,1,2A,2024-10-05,14:00:00,Campo de Hortaleza,XV Hortaleza Rojo,Complutense Cisneros Palito V,33.0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2A,2024-10-05,15:00:00,Campo Alcalá (Luisón Abad),Club Rugby Alcalá,Aeronáuticos,1.0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2A,2024-10-05,16:00:00,Campo de Tres Cantos (Polideportivo de la Luz),Rugby Unión Norte,BigMat Tabanera Lobos Segovia,1.0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2A,2024-10-06,12:00:00,Campo de Getafe (El Bercial),Getafe Club de Rugby,Industriales C SlashMobility Las Rozas,22.0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 02. Calculate results

In [30]:
#Find the unique values of the scoreboard
#Locals and visitors columns
scoreboard_columns_local = r'scoreboard_local_\d{2}_type'
scoreboard_columns_visitor = r'scoreboard_visitor_\d{2}_type'

#Join conditions for filtering in one condition
scoreboard_columns = f'{scoreboard_columns_local}|{scoreboard_columns_visitor}' #Cuidado con meter un espacio entre medias

#Filter scorebodard type columns 
scoreboard_df = matches_df.filter(regex=scoreboard_columns)

#Store the types in array
scoreboard_types = scoreboard_df.stack().dropna().unique()
print(scoreboard_types)

['E' 'T' 'PC' 'D' 'EC']


In [31]:
#The value per action is the next: 
#'E': 5 points
#'T': 2 points
#'PC': 3 points
#'D': 3 points
#'EC': 5 points 

scoreboard_values = [5, 2, 3, 3, 5]

#Create a dictionary to associate action to values
scoreboard_dict = dict(zip(scoreboard_types, scoreboard_values))

print(scoreboard_dict)

{'E': 5, 'T': 2, 'PC': 3, 'D': 3, 'EC': 5}


In [32]:
#Create a column for local points scoreboard and another for visitors
#Evaluate every scoreboard_local/visitor_xx_type and add the value to the new columns created

#Create new column as a series
local_scoreboard_series = None
temp_local_scoreboard_series = None

for column in matches_df.filter(regex=scoreboard_columns_local).columns:
    if local_scoreboard_series is None:
        temp_local_scoreboard_series = matches_df[column].map(scoreboard_dict).fillna(0)
        local_scoreboard_series = temp_local_scoreboard_series.copy()
    else: 
        temp_local_scoreboard_series = matches_df[column].map(scoreboard_dict).fillna(0)
        local_scoreboard_series = local_scoreboard_series + temp_local_scoreboard_series 


#Create new column as a series
visitor_scoreboard_series = None
temp_visitor_scoreboard_series = None

for column in matches_df.filter(regex=scoreboard_columns_visitor).columns:
    if visitor_scoreboard_series is None:
        temp_visitor_scoreboard_series = matches_df[column].map(scoreboard_dict).fillna(0)
        visitor_scoreboard_series = temp_visitor_scoreboard_series.copy()
    else: 
        temp_visitor_scoreboard_series = matches_df[column].map(scoreboard_dict).fillna(0)
        visitor_scoreboard_series = visitor_scoreboard_series + temp_visitor_scoreboard_series 

In [33]:
#Check the results
matches_df['local_scoreboard_points'] = local_scoreboard_series
matches_df['visitor_scoreboard_points'] = visitor_scoreboard_series
matches_df[['local_team', 'local_scoreboard_points', 'visitor_scoreboard_points', 'visitor_team']]

#Results are correct compared to matches reports in pdf

,local_team,local_scoreboard_points,visitor_scoreboard_points,visitor_team
0,C.R Majadahonda,97.0,5.0,Canal Red Vallecas Rugby Union
1,XV Hortaleza Rojo,10.0,67.0,Complutense Cisneros Palito V
2,Club Rugby Alcalá,72.0,14.0,Aeronáuticos
3,Rugby Unión Norte,24.0,17.0,BigMat Tabanera Lobos Segovia
4,Getafe Club de Rugby,10.0,41.0,Industriales C SlashMobility Las Rozas
...,...,...,...,...
68,Rugby Alcorcón,44.0,31.0,Complutense Cisneros Palito V
69,Complutense Cisneros Palito V,31.0,29.0,Canal Red Vallecas Rugby Union
70,Industriales C SlashMobility Las Rozas,38.0,62.0,BigMat Tabanera Lobos Segovia
71,Rugby Alcorcón,23.0,25.0,C.R Majadahonda


# 03. Change jersey number by license number

To handle the dataset with a unique number id for every player. 

In [34]:
#Create two dictionaries (one for local team and other for visitor) for every match with 
#key: player_jersey_number and value: license_number.

#The columns to associate in a dictionary shall be those that contain 'xx_jersey' and 'xx_license'

#Not downcasting since this shall be removed in future versions
pd.set_option('future.no_silent_downcasting', True)

#Loop to run per match (row)
for j in range(len(matches_df)):

    #Lists to store the jersey number and the license number per match
    local_team_jerseys = []
    local_team_license = []
    
    visitor_team_jerseys = []
    visitor_team_license = []  
    
    for i in range(1,24): 
        local_team_jerseys.append(matches_df[f"local_team_player_{i:02d}_jersey"][j])
        local_team_license.append(matches_df[f"local_team_player_{i:02d}_license_number"][j])
    
        visitor_team_jerseys.append(matches_df[f"visitor_team_player_{i:02d}_jersey"][j])
        visitor_team_license.append(matches_df[f"visitor_team_player_{i:02d}_license_number"][j])
    
    #Transform lists into a dictionary
    local_team_jersey_to_license = dict(zip(local_team_jerseys, local_team_license))
    visitor_team_jersey_to_license = dict(zip(visitor_team_jerseys, visitor_team_license))
    
    #Create a filter for the columns that transformation shall be implemented
    local_filter_01 = r'scoreboard_local_\d{2}_player'
    local_filter_02 = r'substitutions_local_\d{2}_player_in'
    local_filter_03 = r'substitutions_local_\d{2}_player_out'
    local_filter_04 = r'cards_local_\d{2}_player'
    
    visitor_filter_01 = r'scoreboard_visitor_\d{2}_player'
    visitor_filter_02 = r'substitutions_visitor_\d{2}_player_in'
    visitor_filter_03 = r'substitutions_visitor_\d{2}_player_out'
    visitor_filter_04 = r'cards_visitor_\d{2}_player'
    
    #Merge all the filter in a general one
    local_filter = f'{local_filter_01}|{local_filter_02}|{local_filter_03}|{local_filter_04}'
    visitor_filter = f'{visitor_filter_01}|{visitor_filter_02}|{visitor_filter_03}|{visitor_filter_04}'

    #Get the columns to filter
    local_columns = matches_df.iloc[j].filter(regex=local_filter)
    visitor_columns = matches_df.iloc[j].filter(regex=visitor_filter)

    #Replace values in filtered columns
    local_replace_columns = local_columns.replace(local_team_jersey_to_license)
    visitor_replace_columns = visitor_columns.replace(visitor_team_jersey_to_license)

    #Asiggn replaces values to corresponding columns in the original dataframe
    matches_df.loc[j, local_replace_columns.index] = local_replace_columns
    matches_df.loc[j, visitor_replace_columns.index] = visitor_replace_columns

# 4. Teams Analysis

## 4.1. 
I would like to answer to the following questions: 

- How many players they have per match?
## 4.2. Statistics 
- How many players they have durig the competition? (later)
- How teams perform during...
    - the first 10 minutes of the match?
    - the first 10 minutes of the second half?
    - the last 20 minutes of the match?

In [35]:
#List every team from competition

teams_list = matches_df[['local_team', 'visitor_team']].stack().unique()

for i, team in enumerate(teams_list): 
    print(i, ": ", team)

0 :  C.R Majadahonda 
1 :  Canal Red Vallecas Rugby Union 
2 :  XV Hortaleza Rojo 
3 :  Complutense Cisneros Palito V 
4 :  Club Rugby Alcalá 
5 :  Aeronáuticos 
6 :  Rugby Unión Norte 
7 :  BigMat Tabanera Lobos Segovia 
8 :  Getafe Club de Rugby 
9 :  Industriales C SlashMobility Las Rozas 
10 :  Rugby Alcorcón 
11 :  Jabatos Móstoles R.C. 


## How many players do teams have per match?



In [36]:
#Calculate the total players per match

#Create twho lists where these values shall be stored
local_team_players = []
visitor_team_players = []

#Run through every match
for i in matches_df.index:

    #Create a series with the players jerssey
    local_lineup = matches_df.iloc[i].filter(regex='local_team_player_\d{2}_jersey')
    visitor_lineup = matches_df.iloc[i].filter(regex='visitor_team_player_\d{2}_jersey')

    #Sum the amout of players per match and append to the lists
    local_team_players.append(local_lineup.notna().sum())
    visitor_team_players.append(visitor_lineup.notna().sum())


matches_df['local_team_players'] = local_team_players
matches_df['visitor_team_players'] = visitor_team_players

In [37]:
columns = ['date', 'local_team', 'local_scoreboard_points', 'visitor_scoreboard_points', 'visitor_team', 'local_team_players', 'visitor_team_players']

matches_df[columns].head()

,date,local_team,local_scoreboard_points,visitor_scoreboard_points,visitor_team,local_team_players,visitor_team_players
0,2024-10-05,C.R Majadahonda,97.0,5.0,Canal Red Vallecas Rugby Union,23,18
1,2024-10-05,XV Hortaleza Rojo,10.0,67.0,Complutense Cisneros Palito V,21,22
2,2024-10-05,Club Rugby Alcalá,72.0,14.0,Aeronáuticos,23,17
3,2024-10-05,Rugby Unión Norte,24.0,17.0,BigMat Tabanera Lobos Segovia,22,22
4,2024-10-06,Getafe Club de Rugby,10.0,41.0,Industriales C SlashMobility Las Rozas,21,21


## 4.2. Statistics about the teams
- Plot their scoreboard results as plot bars

In [ ]:
#To begin the Statistical Analysis it is first necessary to create a dataframe per team

def create_team_df (matches_df, team):
    #Create a dataframe for the team as team and its rival teams, respectively
    as_local_team_df = matches_df[matches_df['local_team'] == team].copy()
    as_visitor_team_df = matches_df[matches_df['visitor_team'] == team].copy()
    
    #Ajust column names
    #In as_local_team_df the word 'local' shall dissapear from column names, while 'visitor' shall be switched to 'rival', 
    #and a new column named 'as_local' with value equals to 1 shall be created and inserted after 'rival_team' column
    as_local_team_df.columns = as_local_team_df.columns.str.replace(r'local_', '', regex=True)
    as_local_team_df.columns = as_local_team_df.columns.str.replace(r'visitor', 'rival', regex=True)
    as_local_team_df.insert(7, 'as_local', 1)
    
    #In as_visitor_team the word 'visitor' shall dissapear from column names, while 'local' shall be switched to 'rival', 
    #and a new column named 'as_local' with value equals to 0 shall be created and insserted after 'rival_team' column. 
    as_visitor_team_df.columns = as_visitor_team_df.columns.str.replace(r'visitor_', '', regex=True)
    as_visitor_team_df.columns = as_visitor_team_df.columns.str.replace(r'local', 'rival', regex=True)
    as_visitor_team_df.insert(7, 'as_local', 0)
    
    #In addition, the columns including 'rival_team' and 'team' from the 'as_visitor_team_df' dataframe need to be 
    #reordered as the columns from 'as_local_team_df'
    as_local_team_columns = as_local_team_df.columns #Get the 'as_local_team_df' columns
    
    #Add every missing columns from 'as_local_team_df' to 'as_visitor_team_df'
    for column in as_local_team_columns: 
        if column not in as_visitor_team_df.columns: 
            as_visitor_team_df[column] = np.nan
    
    as_visitor_team_df = as_visitor_team_df[as_local_team_columns] #Reorder the columns
    
    team_df = pd.concat([as_local_team_df, as_visitor_team_df])
    team_df.sort_values(['date', 'match_time'], inplace=True)
    
    return team_df